# 1) Introducción

La empresa OilyGiant nos ha pedido que encontremos 200 pozos nuevos de petróleo.
Para esto nos han brindado 3 bases de datos que contienen las coordenadas de los pozos actuales y la cantidad en miles de barriles, de petróleo crudo por pozo.
Requieren que nuestro modelo tenga el margen de ganacia más grande, y un riesgo de pérdida menor al 2.5%

# 2) Cargar datos y preparación

La primero que haremos es revisar las bases de datos que nos han proporcionado y la vamos a limpiar la data que encontremos sea necesaria para después ser usada en el análisis.

In [1]:
import pandas as pd #Importar librerías básicas y de estadística para realizar modelo
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from numpy.random import RandomState

In [2]:
#base_0 = pd.read_csv('geo_data_0.csv') #Leer bases de datos
#base_1 = pd.read_csv('geo_data_1.csv')
#base_2 = pd.read_csv('geo_data_2.csv')

base_0 = pd.read_csv('/datasets/geo_data_0.csv')
base_1 = pd.read_csv('/datasets/geo_data_1.csv')
base_2 = pd.read_csv('/datasets/geo_data_2.csv')

print(base_0.info()) #Revisamos info en bases de datos
print()
print(base_0.isna().sum()) #Revisamos datos nulos en nuestro DataFrame
print()
print(base_0.duplicated().sum())

#No parece haber datos nulos o faltantes, al ser coordenadas podemos asumira que toda la información está lista para ser usada.
#Este es el mismo caso para las otras bases, así que no será necesaria limpiar la base, se puede usar tal como está.
#Ya podemos empezar a manipular la información para crear el modelo.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

0


# 3) Entrenar primer modelo

Ahora que ya tenemos la data lista la vamos a dividi para poder entrenar el modelo.
Vamos a asesosrar como va proviendo los resultados y saber si debemos de modificar el modelo para mejorar.
También haremos evaluaciones de riesgo para poder llegar a margen esperado.

In [3]:
train_base_0, valid_base_0 = train_test_split(base_0, test_size=0.25, random_state=12345) #Dividimos nuestras bases en muestreos para entrenar y validar el modelo.
train_base_1, valid_base_1 = train_test_split(base_1, test_size=0.25, random_state=12345)
train_base_2, valid_base_2 = train_test_split(base_2, test_size=0.25, random_state=12345)

feat_train_b0 = train_base_0.drop(['product', 'id'] , axis=1) #Dividimos nuestras bases en nuestras características  de entrenamiento y validación de objetivos.
target_train_b0 = train_base_0['product']            #En este caso buscamos la mayor cantidad  de barriles, así que la columna 'product' con la cantidad de barriles es nuestro objetivo a alcanzar

feat_valid_b0 = valid_base_0.drop(['product', 'id'] , axis=1)
target_valid_b0 = valid_base_0['product']

feat_train_b1 = train_base_1.drop(['product', 'id'] , axis=1)
target_train_b1 = train_base_1['product']

feat_valid_b1 = valid_base_1.drop(['product', 'id'] , axis=1)
target_valid_b1 = valid_base_1['product']

feat_train_b2 = train_base_2.drop(['product', 'id'] , axis=1)
target_train_b2 = train_base_2['product']

feat_valid_b2 = valid_base_2.drop(['product', 'id'] , axis=1)
target_valid_b2 = valid_base_2['product']

In [4]:
model_0 = LinearRegression()                   #Entrenamos los modelos con los conjuntos de entrenamiento.
model_0.fit(feat_train_b0, target_train_b0)    

model_1 = LinearRegression()
model_1.fit(feat_train_b1, target_train_b1)

model_2 = LinearRegression()
model_2.fit(feat_train_b2, target_train_b2)

predicted_valid_b0 = model_0.predict(feat_valid_b0) #Guardamos las predicciones del modelo para usar distintos métodos de validación.
predicted_valid_b1 = model_1.predict(feat_valid_b1)
predicted_valid_b2 = model_2.predict(feat_valid_b2)

In [5]:
print('Validación base_0')
print(f"Promedio de miles de barriles por pozo predicción base_0:", predicted_valid_b0.mean()) #Mostramos cantidad de miles de barriles en promedio dentro de la predicción.
print(f'Error cuadrático medio base_0:', mean_squared_error(target_valid_b0, predicted_valid_b0)) #Mostramos el error cuadrático de nuestra predicción vs. el conjunto de validación.
print(f'Raíz del error cuadrático medio base_0:', (mean_squared_error(target_valid_b0, predicted_valid_b0) **  0.5)) #Sacamos la raíz para conocer nuestra diferencia en cantidad de barriles.
print()
print('Validación base_1')
print(f"Promedio de miles de barriles por pozo predicción base_1:", predicted_valid_b1.mean())
print(f'Error cuadrático medio base_1:', mean_squared_error(target_valid_b1, predicted_valid_b1))
print(f'Raíz del error cuadrático medio base_1:', (mean_squared_error(target_valid_b1, predicted_valid_b1) **  0.5))
print()
print('Validación base_2')
print(f"Promedio de miles de barriles por pozo predicción base_2:", predicted_valid_b2.mean())
print(f'Error cuadrático medio base_2:', mean_squared_error(target_valid_b2, predicted_valid_b2))
print(f'Raíz del error cuadrático medio base_2:', (mean_squared_error(target_valid_b2, predicted_valid_b2) **  0.5))
print()
print('Conclusiones resultados RMSE: Aqui observamos que la base 1 tiene el error más pequeño, podemos suponer que en esta base, los datos se comportan de una manera ordinaria.\nLas bases 0 y 2, tienen un error elevado, el comportamiento en estas bases es anormal.\nApesar de eso, estas bases muestran una cantidad promedio de barriles por pozo mayor.\nAsí que la inversión en estos sitios puede ser de mayor riesgo pero mayor ganancia')

Validación base_0
Promedio de miles de barriles por pozo predicción base_0: 92.59256778438035
Error cuadrático medio base_0: 1412.2129364399243
Raíz del error cuadrático medio base_0: 37.5794217150813

Validación base_1
Promedio de miles de barriles por pozo predicción base_1: 68.728546895446
Error cuadrático medio base_1: 0.7976263360391157
Raíz del error cuadrático medio base_1: 0.893099286775617

Validación base_2
Promedio de miles de barriles por pozo predicción base_2: 94.96504596800489
Error cuadrático medio base_2: 1602.3775813236196
Raíz del error cuadrático medio base_2: 40.02970873393434

Conclusiones resultados RMSE: Aqui observamos que la base 1 tiene el error más pequeño, podemos suponer que en esta base, los datos se comportan de una manera ordinaria.
Las bases 0 y 2, tienen un error elevado, el comportamiento en estas bases es anormal.
Apesar de eso, estas bases muestran una cantidad promedio de barriles por pozo mayor.
Así que la inversión en estos sitios puede ser de m

# 4) Validación de Ganancias

Ya conocemos cuales son los errores de desviación y los posibles promedios por pozo.
Conociendo esta información veremos si podemos obtener ganacias de estos pozos.
Sabemos que el costo total para la realización de los pozos y la extracción es de 100 millones de dólares.
También sabemos que son 200 pozos nuevos los que se quieren crear.
Teniendo esto en mente, dividimos 100 millones / 200 = 0.5 millones por pozo.
En promedio todos los pozos deben factirar 0.5 millones de dolares por pozo para poder cubrir costos y evitar pérdidas.
Un barril de materia prima genera 4.5 USD, si escalamos eso a nuestra medida de miles, entonces serían 4,500 USD por mil barriles de petróleo.

In [6]:
pozos_200_0 = pd.Series(predicted_valid_b0).sort_values(ascending=False)[:200]
pozos_200_1 = pd.Series(predicted_valid_b1).sort_values(ascending=False)[:200]
pozos_200_2 = pd.Series(predicted_valid_b2).sort_values(ascending=False)[:200]

In [7]:
print('Validación ganacias pozos base_0')
print(f'Predicción de promedio de ganancia por pozo de base_0 en ganancias USD:', int((predicted_valid_b0 * 4500).mean())) #Validamos la ganacia promedio por pozo, consideramos la unidad en miles de barriles así que multiplicamos por 4.5K para para conocer el ingreso promedio.
print(f'Predicción de ganacia de 200 pozos en base_0 USD:', int((pozos_200_0 * 4500).sum()))                       #Validamos la ganancia total según la predicción
if int(((pozos_200_0 * 4500).sum()) > 100000000):                                                                   #Comparamos los cotos vs. las ganancias totales de los nuevos pozos para conocer el retorno de inversión.
    print(f'La ganancia supera a los costos por:', int(((pozos_200_0 * 4500).sum()) - 100000000))
else:
    print(f'La pérdida super a los costos por:', int(((pozos_200_0 * 4500).sum()) - 100000000))
print()
print('Validación ganacias pozos base_1')
print(f'Predicción de promedio de ganancia por pozo de base_1 en ganancias USD:', int((pozos_200_1 * 4500).mean()))
print(f'Predicción de ganacia de 200 pozos en base_1 USD:', int((predicted_valid_b1 * 4500).sum()))
if int(((pozos_200_1 * 4500).sum()) > 100000000):
    print(f'La ganancia supera a los costos por:', int(((pozos_200_1 * 4500).sum()) - 100000000))
else:
    print(f'La pérdida super a los costos por:', int(((pozos_200_1 * 4500).sum()) - 100000000))
print()
print('Validación ganacias pozos base_2')
print(f'Predicción de promedio de ganancia por pozo de base_2 en ganancias USD:', int((pozos_200_2 * 4500).mean()))
print(f'Predicción de ganacia de 200 pozos en base_2 USD:', int((pozos_200_2 * 4500).sum()))
if int(((pozos_200_2 * 4500).sum()) > 100000000):
    print(f'La ganancia supera a los costos por:', int(((pozos_200_2 * 4500).sum()) - 100000000))
else:
    print(f'La pérdida super a los costos por:', int(((pozos_200_2 * 4500).sum()) - 100000000))
print()
print('Todas las predicciones muestran una ganancia total supera los costos de inversión.\nSi observamos la ganacia promedio por pozo de cada base, podemos ver que ninguno super los 0.5 millones que buscamos.\nSabiendo esto, vamos a elegir de cada base los pozos con mayor cantidad de barriles.\nDebemos elegir los pozos que representen un mayor ingreso con el menor riesgo.')

Validación ganacias pozos base_0
Predicción de promedio de ganancia por pozo de base_0 en ganancias USD: 416666
Predicción de ganacia de 200 pozos en base_0 USD: 139960488
La ganancia supera a los costos por: 39960488

Validación ganacias pozos base_1
Predicción de promedio de ganancia por pozo de base_1 en ganancias USD: 624285
Predicción de ganacia de 200 pozos en base_1 USD: 7731961525
La ganancia supera a los costos por: 24857120

Validación ganacias pozos base_2
Predicción de promedio de ganancia por pozo de base_2 en ganancias USD: 666087
Predicción de ganacia de 200 pozos en base_2 USD: 133217543
La ganancia supera a los costos por: 33217543

Todas las predicciones muestran una ganancia total supera los costos de inversión.
Si observamos la ganacia promedio por pozo de cada base, podemos ver que ninguno super los 0.5 millones que buscamos.
Sabiendo esto, vamos a elegir de cada base los pozos con mayor cantidad de barriles.
Debemos elegir los pozos que representen un mayor ingres

# 5) Submuestreo

Ahora debemos de seleccionar los pozos con mayor retorno de inversión, estos deben de provenir de cada una de las bases de datos provistas.
Para esto vamos a utilizar el 'bootstrapping' realizando un submuestreo para encontrar a los pozos con mayor cantidad de barriles dentro de un intervalo de confianza.
Este intervalo de confianza está delimitado por la cantidad de barriles mínima que se buscan por pozo, en esta caso es de 111.1 barriles por pozo.

In [8]:
#Ahora vamos a hacer un submuestreo para encontrar los pozos con mayor cantidad de barriles.
#Para esto vamos a ordenar los datos de manera descendente y dividirlos en cuartiles

def bootstrap(predicciones, n_simulaciones=1000):
    valores = []
    for _ in range(n_simulaciones):
        muestra = predicciones.sample(n=500, replace=True)
        mejores_pozos = muestra.sort_values(ascending=False)[:200]
        ganancia_muestra = mejores_pozos.sum() * 4500
        valores.append(ganancia_muestra)
    valores = pd.Series(valores)
    intervalo_de_confianza = np.percentile(valores, [2.5, 97.5])
    riesgo_perdida = (valores < 100000000).mean() * 100
    return valores.mean(), intervalo_de_confianza, riesgo_perdida

In [9]:
media_0, intervalo_0, riesgo_0 = bootstrap(pd.Series(predicted_valid_b0))
media_1, intervalo_1, riesgo_1 = bootstrap(pd.Series(predicted_valid_b1))
media_2, intervalo_2, riesgo_2 = bootstrap(pd.Series(predicted_valid_b2))

print(f"Región 0:\nBeneficio promedio: ${media_0:.2f}\nIntervalo de confianza 95%: {intervalo_0}\nRiesgo de pérdida: {riesgo_0:.2f}%\n")
print(f"Región 1:\nBeneficio promedio: ${media_1:.2f}\nIntervalo de confianza 95%: {intervalo_1}\nRiesgo de pérdida: {riesgo_1:.2f}%\n")
print(f"Región 2:\nBeneficio promedio: ${media_2:.2f}\nIntervalo de confianza 95%: {intervalo_2}\nRiesgo de pérdida: {riesgo_2:.2f}%\n")
print("En este caso las tres regiones tienen un riesgo de pérdida menor al 1%.\nTeniendo en consideración toda la información que hemos analisado hasta el momento.\nDiría que la Región 2 es la más llamativa.\nPresenta un riesgo bajo y anteriormente demostraro tener un promedio alto de barriles por pozo.\nTambién presentó el promedio de ganancia promedio por pozo más alto.\nEsto es una sugerencia, OilyGiant tendría que asesorar cual es la región que mejor les acomode basado en las métricas presentadas.")

Región 0:
Beneficio promedio: $103489005.67
Intervalo de confianza 95%: [1.01164395e+08 1.05666923e+08]
Riesgo de pérdida: 0.30%

Región 1:
Beneficio promedio: $104634571.88
Intervalo de confianza 95%: [1.00696128e+08 1.08953170e+08]
Riesgo de pérdida: 1.30%

Región 2:
Beneficio promedio: $102725451.83
Intervalo de confianza 95%: [1.00889256e+08 1.04530749e+08]
Riesgo de pérdida: 0.30%

En este caso las tres regiones tienen un riesgo de pérdida menor al 1%.
Teniendo en consideración toda la información que hemos analisado hasta el momento.
Diría que la Región 2 es la más llamativa.
Presenta un riesgo bajo y anteriormente demostraro tener un promedio alto de barriles por pozo.
También presentó el promedio de ganancia promedio por pozo más alto.
Esto es una sugerencia, OilyGiant tendría que asesorar cual es la región que mejor les acomode basado en las métricas presentadas.
